# 乱数を使って保持器を作る

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(0)
params = np.random.randn(100, 10) / 3
params[params<-1] = -1
params[params>1] = 1
params[:2]

In [ ]:
params.max(), params.min()

In [ ]:
r0 = lambda dr: 2.35e-3 + dr
r1 = lambda dr: 2.85e-3 + dr

r0(0.25e-3), r1(-0.25e-3)

In [ ]:
r0s = r0(params[:, 0] * 0.2e-3)
r1s = r1(params[:, 1] * 0.2e-3)
drs = r1s - r0s

drs.min() * 1e3, drs.max() * 1e3, r0s.min() * 1e3, r1s.max() * 1e3


In [ ]:
h0 = lambda dh: 0.93e-3 + dh
h1 = lambda dh: 2.10e-3 + dh

h0s = h0(params[:, 2] * 0.2e-3)
h1s = h1(params[:, 3] * 0.2e-3)

h0s.max() * 1e3, h0s.min() * 1e3, h1s.max() * 1e3, h1s.min() * 1e3

In [ ]:
bevel = lambda r: 0.1 * drs * r

bevels = bevel(params[:, 4]+2)

(bevels / drs).max(), (bevels / drs).min()

In [ ]:
pocket_r = lambda dr: 0.825e-3 + dr

pocket_rs = pocket_r(params[:, 5] * 1e-6)
pocket_rs.min() * 1e3, pocket_rs.max() * 1e3

In [ ]:
neck_x = lambda dx: 1.70e-3 + dx

neck_xs = neck_x(params[:, 6] * 20e-6)
neck_xs.min() * 1e3, neck_xs.max() * 1e3

In [ ]:
neck_r = lambda dr: 1.20e-3 + dr

neck_rs = neck_r(params[:, 7] * 20e-6)
neck_rs.min() * 1e3, neck_rs.max() * 1e3


In [ ]:
neck_h = lambda dh: 2.45e-3 + dh

neck_hs = neck_h(params[:, 8] * 100e-6)
neck_hs.min() * 1e3, neck_hs.max() * 1e3


In [ ]:
neck_dh = lambda r: (neck_hs - h1s) * r * 0.2

neck_dhs = neck_dh(params[:, 9]+2)
neck_dhs.min() * 1e3, neck_dhs.max() * 1e3


In [ ]:
df = pd.DataFrame({
    'r0': r0s,
    'r1': r1s,
    'h0': h0s,
    'h1': h1s,
    'bevel': bevels,
    'pocket_r': pocket_rs,
    'neck_x': neck_xs,
    'neck_r': neck_rs,
    'neck_h': neck_hs,
    'neck_dh': neck_dhs
})

df.to_csv('../data/random_params.csv', index=False)

In [ ]:
filenames = [f'data/random/Tetra_linspace{i}.vtu' for i in range(len(params))]
filenames[-2]


In [ ]:
command_args = lambda i: f'{filenames[i]} {r0s[i]} {r1s[i]} {h0s[i]} {h1s[i]} {bevels[i]} {pocket_rs[i]} {neck_xs[i]} {neck_rs[i]} {neck_hs[i]} {neck_dhs[i]} 1000'

command_args(10)

# '../data/Tetra_linspace10.vtu 0.0024755433798037503 0.002760149395923837 0.0008453010001009512 0.002164626447210534 4.579193726579163e-05 0.0008256478737285498 0.0016972425401282683 0.0011950169679237283 0.002514098067549346 0.0001742818416754026 1000'

In [ ]:
import subprocess

command = "cd ../; cargo run --release "
# subprocess.run(command + command_args(0), shell=True)

for i in range(len(params)):
    subprocess.run(command + command_args(i), shell=True)
    print(f'Finished {i} of {len(params)}')
